## General assumptions about the data
   * SUTVA - We assume there's no spillover effect, and marriage "treatment" is the same across people. The latter is fixed such that marriage = having legal notice, but the effect is hard to justify it is equal among everyone. Nevertheless, we will assume so. 
   * Randomness - The survey users are randomly chosen. Even if we assume the survey users are chosen randomly (to represent the US population), the dropout may not be random, which may have some bias. 

## Notes
   * This is using logitudinal data however it's not really a panel data analysis since we are not comparing two points in time (as data is not available)
   * We will mainly use the 2019 data as it's pre-COVID, but may look at 2021 data for comparison
   * To reduce dependency between rounds (i.e. need to run previous rounds prior to running next round), some code may be redundant between rounds
   * At some point PCA can be used but is not being done here as I have not really studied PCA yet. 
   

## Rough course of action

1. Decide on datapoints 
2. Prepare 

## Imports

In [5]:
import pandas as pd
from io import StringIO
import re
from itables import init_notebook_mode
from itables import show

import math
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
import statsmodels.formula.api as smf

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from scipy.stats import chi2_contingency
from scipy.stats import ks_2samp
from scipy.stats import mannwhitneyu

import pickle

## Data preprocess

In [7]:
target_year = 2019
target_outcome = 'CESD_above_8'

In [8]:
def better_colnames(df, from_col: str, to_cols: list[str]  = ['Description','Year']):
    ''' Helper function to rename the columns in the (NLS data) df into something readable/helpful, based on data in codebook_df
    '''
    #Create dataframe with Ref column and the desired new column names based on input
    col_mapping = pd.concat([codebook_df[from_col],codebook_df[to_cols[0]].str.cat(codebook_df[to_cols[1:]],sep='_')],axis=1)    
    col_mapping.columns.values[1] = '_'.join(to_cols)
    #Convert the mapping dataframe to dictionary and rename columns
    return(df.rename(columns = col_mapping.set_index(from_col).squeeze().to_dict()))

In [9]:
tidy_df= pd.read_pickle('tidy_df.pkl')
main_df= pd.read_pickle('main_df.pkl')
codebook_df= pd.read_pickle('codebook_df.pkl')
mental_health_df= pd.read_pickle('mental_health_df.pkl')
codebook_df= pd.read_pickle('codebook_df.pkl')
cohab_mhealth_df = pd.read_pickle('cohab_mhealth_df.pkl')
source_df = pd.read_pickle('source_df.pkl')

In [10]:
#Model data creation

tidy_df_cols = {
    'uid': 'uid',
    'sex': 'sex',
    'age_1996': 'age_1996',
    'hispanic': 'hispanic',
    'race': 'race',
    'CV_URBAN_RURAL': 'urban'
}
source_df_cols = {

}
    

#Initially add columns from tidy_df 
model_data_df = tidy_df.loc[tidy_df['Year'] == target_year,list(tidy_df_cols.keys()),]

#Add source_df columns
#model_data_df = model_data_df.merge(source_df.loc[:,['uid'] + list(source_df_cols.keys())],on='uid')


#Add derived columns from previous work
cohab_mhealth_cols = ['uid','Rxx_CESD_SCORE_(x_ITEM)','married',target_outcome]
model_data_df = model_data_df.merge(cohab_mhealth_df.loc[cohab_mhealth_df['Year']==2019,cohab_mhealth_cols],on='uid')
model_data_df = model_data_df.rename(columns={'Rxx_CESD_SCORE_(x_ITEM)':'CESD'})

#converting the main treatment to int
#Since there are three states (Married, Not married but married before, Never married), we will create two separate columns 
model_data_df['married_vs_not_married'] = (model_data_df['married'] == 'Yes').astype(int)
model_data_df['married_vs_never_married'] = (model_data_df['married'] == 'Yes').astype(int)
#For married VS never married, the "Married but married before" users will be marked as NaN
model_data_df.loc[model_data_df['married']=='No',['married_vs_never_married']] = np.nan

model_data_df.rename(columns=tidy_df_cols,inplace=True)
#model_data_df.rename(columns=source_df_cols,inplace=True)

In [11]:
# Currently a lot of the columns are stored as sparse data. Some models choke as they don't handle such datatype. 
# Therefore, changing them to dense

def dense_columns(col):
    if (isinstance(col.dtype, pd.SparseDtype)): 
        return col.sparse.to_dense()
    else:
        return col
    
model_data_df = model_data_df.apply(dense_columns,axis=0)

In [12]:
#Create BMI columns. Since height and weight not mentioned every year, need to pull from last known

#Get index for last known weight and height 
last_known_wgt_idx = tidy_df.loc[(tidy_df['YSAQ_000B'].notnull()) & (tidy_df['YSAQ_000B'] > 0),:].groupby('uid')['YSAQ_000B'].idxmax()
last_known_hgt_idx = tidy_df.loc[(tidy_df['YSAQ_000A000001'].notnull()) & (tidy_df['YSAQ_000A000001'] > 0)
                                 & (tidy_df['YSAQ_000A000002'].notnull()) & (tidy_df['YSAQ_000A000002'] >= 0)
                                 ,:].groupby('uid')['YSAQ_000A000001'].idxmax()


#Using the values above, get bmi
bmi_df = tidy_df.loc[last_known_wgt_idx,['uid','YSAQ_000B']]
bmi_df = bmi_df.merge(tidy_df.loc[last_known_hgt_idx,['uid','YSAQ_000A000001','YSAQ_000A000002']],on='uid',how='left')
bmi_df['bmi'] = bmi_df['YSAQ_000B'] / ((bmi_df['YSAQ_000A000001'] * 12 + bmi_df['YSAQ_000A000002']) ** 2) * 703

model_data_df.drop(['bmi','bmi_bin'],inplace=True,errors='ignore')
model_data_df = model_data_df.merge(bmi_df[['uid','bmi']], on='uid',how='left')

#Imputing bmi based on US average. Technically the average is different per male and female, but it's slight difference
# (29.1 VS 29.6) so just using the same value.
model_data_df['bmi'] = model_data_df['bmi'].fillna(29.4)

#Create bin
model_data_df['bmi_bin'] = ''
model_data_df.loc[model_data_df['bmi'] < 18.5,'bmi_bin'] = 'under'
model_data_df.loc[model_data_df['bmi'].between(18.5, 25.0,inclusive='left'),'bmi_bin'] = 'normal'
model_data_df.loc[model_data_df['bmi'].between(25.0, 29.0,inclusive='left'),'bmi_bin'] = 'over'
model_data_df.loc[model_data_df['bmi'] > 29.0,'bmi_bin'] = 'obese'
model_data_df['bmi_bin'] = model_data_df['bmi_bin'].astype('category')

In [48]:
#Flipping urban variable values (0:rural 1:urban 2:unknown) to somewhat more ordinal (0:unknown 1:rural 2:urban)

model_data_df['urban'] = model_data_df['urban'].map({0:1,1:2,2:0})

In [66]:
model_data_df.sparse

AttributeError: Can only use the '.sparse' accessor with Sparse data.

In [52]:
model_data_df.describe()

TypeError: cannot perform std with type Sparse[float64, nan]

## Round 0 - Logistic regression w/ only treatment variable
   * Use logistic regression w/ married_vs_never_married only
   * Odds increase = 2.3x, w/ p-value < 0.01

In [18]:
temp_model_data_df = model_data_df.copy()
temp_model_data_df.dropna(axis=0,subset='married_vs_never_married',inplace=True)

In [20]:
formula = target_outcome +  ' ~ married_vs_never_married'
print(formula)
# Fit the logistic regression model using GLM
model = smf.glm(formula=formula, data=temp_model_data_df, family=sm.families.Binomial()).fit()

# Print the summary of the model
print(model.summary())
print(f'Log odds increase when married: {math.exp(model.params["married_vs_never_married"])} ')
print(f'P-value: {model.pvalues["married_vs_never_married"]} ')


CESD_above_8 ~ married_vs_never_married
                               Generalized Linear Model Regression Results                               
Dep. Variable:     ['CESD_above_8[False]', 'CESD_above_8[True]']   No. Observations:                 5835
Model:                                                       GLM   Df Residuals:                     5833
Model Family:                                           Binomial   Df Model:                            1
Link Function:                                             Logit   Scale:                          1.0000
Method:                                                     IRLS   Log-Likelihood:                -1700.2
Date:                                           Wed, 07 Aug 2024   Deviance:                       3400.5
Time:                                                   08:36:06   Pearson chi2:                 5.83e+03
No. Iterations:                                                6   Pseudo R-squ. (CS):            0.01333
Covari

## Round 1 - Logistic regression w/ few demographics
   * Use logistic regression w/ age, race-related & sex as additional explanatory variables
   * Additional assumptions
      * No unconfounding variables (HAHAHA...) 
      * Linearity of logit - We should check this but not checking for this round as the above assumption doesn't really hold anyways
   * __Result__
      * Odds increase: 2.5x
      * p-value: < 0.01

In [23]:
temp_model_data_df = model_data_df.copy()
temp_model_data_df.dropna(axis=0,subset='married_vs_never_married',inplace=True)

In [24]:
#formula creation

Xs_numerical = ['age_1996']
Xs_categorical = ['hispanic','sex','race']

formula = target_outcome +  ' ~ married_vs_never_married + ' +  (' + '.join(Xs_numerical)) + ' + C(' + (') + C(').join(Xs_categorical) + ')'
print(formula)

CESD_above_8 ~ married_vs_never_married + age_1996 + C(hispanic) + C(sex) + C(race)


In [25]:
# Fit the logistic regression model using GLM
model = smf.glm(formula=formula, data=temp_model_data_df, family=sm.families.Binomial()).fit()

# Print the summary of the model
print(model.summary())
print(f'Log odds increase when married: {math.exp(model.params["married_vs_never_married"])} ')
print(f'P-value: {model.pvalues["married_vs_never_married"]} ')


                               Generalized Linear Model Regression Results                               
Dep. Variable:     ['CESD_above_8[False]', 'CESD_above_8[True]']   No. Observations:                 5835
Model:                                                       GLM   Df Residuals:                     5822
Model Family:                                           Binomial   Df Model:                           12
Link Function:                                             Logit   Scale:                          1.0000
Method:                                                     IRLS   Log-Likelihood:                -1676.9
Date:                                           Wed, 07 Aug 2024   Deviance:                       3353.8
Time:                                                   08:36:07   Pearson chi2:                 5.80e+03
No. Iterations:                                               20   Pseudo R-squ. (CS):            0.02119
Covariance Type:                              

## Round 2 - Logistic regression w/ more fields
   * Use logistic regression w/ added fields:
      * BMI in bins (underweight, normal, over, obese)
      * Living in urban or rural place
      * Household income at survey year
      * Highest degree CVC_HIGHEST_DEGREE_EVER
      * Kids
      * Number of people in household
   * Additional assumptions
      * No unconfounding variables (HAHAHA...) 
      * Linearity of logit - We should check this but not checking for this round as the above assumption doesn't really hold anyways
   * __Result__
      * Odds increase: 2.5x
      * p-value: < 0.01

In [29]:
temp_model_data_df = model_data_df.copy()
temp_model_data_df.dropna(axis=0,subset='married_vs_never_married',inplace=True)

In [30]:
#formula creation

Xs_numerical = ['age_1996',]
Xs_categorical = ['hispanic','sex','race','bmi_bin','urban']

formula = target_outcome +  ' ~ married_vs_never_married + ' +  (' + '.join(Xs_numerical)) + ' + C(' + (') + C(').join(Xs_categorical) + ')'
print(formula)

CESD_above_8 ~ married_vs_never_married + age_1996 + C(hispanic) + C(sex) + C(race) + C(bmi_bin) + C(urban)


In [45]:
# Fit the logistic regression model using GLM
model = smf.glm(formula=formula, data=temp_model_data_df, family=sm.families.Binomial()).fit()
#model = smf.glm(formula=formula, data=temp_model_data_df, family=sm.families.Binomial()).fit_regularized)


# Print the summary of the model
print(model.summary())
print(f'Log odds increase when married: {math.exp(model.params["married_vs_never_married"])} ')
print(f'P-value: {model.pvalues["married_vs_never_married"]} ')


                               Generalized Linear Model Regression Results                               
Dep. Variable:     ['CESD_above_8[False]', 'CESD_above_8[True]']   No. Observations:                 5835
Model:                                                       GLM   Df Residuals:                     5815
Model Family:                                           Binomial   Df Model:                           19
Link Function:                                             Logit   Scale:                          1.0000
Method:                                                     IRLS   Log-Likelihood:                -1671.9
Date:                                           Wed, 07 Aug 2024   Deviance:                       3343.9
Time:                                                   08:47:41   Pearson chi2:                 5.78e+03
No. Iterations:                                               20   Pseudo R-squ. (CS):            0.02285
Covariance Type:                              

## RAndom

In [ ]:
show(tidy_df[['CV_URBAN_RURAL']])

In [ ]:
#Hot encode columns & keep the column names on the side for modeling

hot_encode_binary = ['sex']
hot_encode_others = ['race']

# For binary, we'll keep only one column. For others, we won't drop the first column
temp_encoded_binaries_df = pd.get_dummies(model_data_df[hot_encode_binary], columns=hot_encode_binary,drop_first=True)
temp_encoded_nonbinaries_df = pd.get_dummies(model_data_df[hot_encode_others], columns=hot_encode_others)


hot_encoded_columns = temp_encoded_binaries_df.columns.to_list() + temp_encoded_nonbinaries_df.columns.to_list()

temp_model_data_df = model_data_df.drop(hot_encode_binary + hot_encode_others, axis=1)
temp_model_data_df = pd.concat([temp_model_data_df,temp_encoded_binaries_df, temp_encoded_nonbinaries_df],axis=1)